In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from bert_score import score as bert_score
from bert_score import BERTScorer
import torch

# Load the BERT model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_model = AutoModel.from_pretrained("bert-base-multilingual-cased")
## Initialize the BERTScorer for multilingual BERT or a Japanese-specific BERT
scorer = BERTScorer(model_type="bert-base-multilingual-cased", lang="ja", device='cuda' if torch.cuda.is_available() else 'cpu')

def cosine_similarity_calc(str1, str2):
    # Initialize MeCab Tokenizer
    #tokenizer = MeCab.Tagger("-Owakati")
    
    # Tokenize and encode the strings
    inputs_1 = tokenizer(str1, return_tensors='pt', truncation=True, padding=True)
    #print(str1)
    #print(str(inputs_1))
    inputs_2 = tokenizer(str2, return_tensors='pt', truncation=True, padding=True)
    #print(str2)
    #print(str(inputs_2))
    
    # Get embeddings from BERT model
    with torch.no_grad():
        outputs_1 = bert_model(**inputs_1)
        outputs_2 = bert_model(**inputs_2)
    
    # Get the embeddings for the [CLS] token
    # This represents the pooled output for each sentence
    embedding_1 = outputs_1.last_hidden_state[:, 0, :].squeeze().numpy()
    embedding_2 = outputs_2.last_hidden_state[:, 0, :].squeeze().numpy()

    # Reshape the embeddings to 2D arrays (1, -1) to ensure proper input to cosine_similarity
    embedding_1 = embedding_1.reshape(1, -1)
    embedding_2 = embedding_2.reshape(1, -1)

    print(embedding_1)
    print(embedding_2)
    
    # Calculate Cosine similarity
    cos_sim = cosine_similarity(embedding_1, embedding_2)
    return cos_sim[0, 0]

/home/ubuntu/anaconda3/envs/reflexion/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dens

In [9]:
import numpy as np
def cosine_similarity_calc_word_mean(str1, str2):
    
    def get_mean_embedding(sentence):
        # Tokenize and encode the sentence
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        
        # Get embeddings from BERT model
        with torch.no_grad():
            outputs = bert_model(**inputs)
        
        # Get the embeddings for all tokens
        embeddings = outputs.last_hidden_state.squeeze(0).numpy()
        
        # Calculate the mean of the embeddings (mean pooling)
        mean_embedding = np.mean(embeddings, axis=0)
        
        return mean_embedding
    
    # Calculate mean embeddings for both strings
    mean_embedding_1 = get_mean_embedding(str1)
    mean_embedding_2 = get_mean_embedding(str2)
    
    # Reshape the embeddings to 2D arrays (1, -1) to ensure proper input to cosine_similarity
    mean_embedding_1 = mean_embedding_1.reshape(1, -1)
    mean_embedding_2 = mean_embedding_2.reshape(1, -1)
    
    # Calculate Cosine similarity
    cos_sim = cosine_similarity(mean_embedding_1, mean_embedding_2)
    
    return cos_sim[0, 0]

In [4]:
keywords = [
    "ソニーのミラーレス",
    "sony ミラーレス",
    "sonyミラーレス",
    "ソニー ミラーレス 一眼",
    "ソニー カメラ　ミラーレス 一眼",
    "ミラーレス ソニー",
    "ミラーレス　Sony",
    "ミラーレス　Sony"
]

# Initialize an empty dictionary to store the results
similarity_scores = {}

# Loop through each combination of keywords
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        kw1 = keywords[i]
        kw2 = keywords[j]
        # Calculate cosine similarity
        score = cosine_similarity_calc_word_mean(kw1, kw2)
        # Store the result in the dictionary
        similarity_scores[(kw1, kw2)] = score
        print(f"Cosine Similarity (word embedding mean)between '{kw1}' and '{kw2}': {score}")



Cosine Similarity between 'ソニーのミラーレス' and 'sony ミラーレス': 0.7652407884597778
Cosine Similarity between 'ソニーのミラーレス' and 'sonyミラーレス': 0.7750669121742249
Cosine Similarity between 'ソニーのミラーレス' and 'ソニー ミラーレス 一眼': 0.7909582853317261
Cosine Similarity between 'ソニーのミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.7931433320045471
Cosine Similarity between 'ソニーのミラーレス' and 'ミラーレス ソニー': 0.8153504729270935
Cosine Similarity between 'ソニーのミラーレス' and 'ミラーレス　Sony': 0.6884340047836304
Cosine Similarity between 'ソニーのミラーレス' and 'ミラーレス　Sony': 0.6884340047836304
Cosine Similarity between 'sony ミラーレス' and 'sonyミラーレス': 0.9181486964225769
Cosine Similarity between 'sony ミラーレス' and 'ソニー ミラーレス 一眼': 0.7477536201477051
Cosine Similarity between 'sony ミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.7243062257766724
Cosine Similarity between 'sony ミラーレス' and 'ミラーレス ソニー': 0.7729085683822632
Cosine Similarity between 'sony ミラーレス' and 'ミラーレス　Sony': 0.7402163743972778


Cosine Similarity between 'sony ミラーレス' and 'ミラーレス　Sony': 0.7402163743972778
Cosine Similarity between 'sonyミラーレス' and 'ソニー ミラーレス 一眼': 0.7049102783203125
Cosine Similarity between 'sonyミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.7149333953857422
Cosine Similarity between 'sonyミラーレス' and 'ミラーレス ソニー': 0.7229804992675781
Cosine Similarity between 'sonyミラーレス' and 'ミラーレス　Sony': 0.7093991041183472
Cosine Similarity between 'sonyミラーレス' and 'ミラーレス　Sony': 0.7093991041183472
Cosine Similarity between 'ソニー ミラーレス 一眼' and 'ソニー カメラ　ミラーレス 一眼': 0.9431635141372681
Cosine Similarity between 'ソニー ミラーレス 一眼' and 'ミラーレス ソニー': 0.778666615486145
Cosine Similarity between 'ソニー ミラーレス 一眼' and 'ミラーレス　Sony': 0.6932469606399536
Cosine Similarity between 'ソニー ミラーレス 一眼' and 'ミラーレス　Sony': 0.6932469606399536
Cosine Similarity between 'ソニー カメラ　ミラーレス 一眼' and 'ミラーレス ソニー': 0.7496322989463806
Cosine Similarity between 'ソニー カメラ　ミラーレス 一眼' and 'ミラーレス　Sony': 0.6856372356414795
Cosine Similarity between 'ソニー カメラ　ミラーレス 一眼' and 'ミラーレス　Sony': 

In [10]:
keywords = [
    "ソニーのミラーレス",
    "一眼レフカメラ",
    "ソニー アルファ9",
    "アルファ7r",
    "ソニー カメラ　ミラーレス 一眼",
    "加賀屋の女将",
    "sonyカメラ新製品",
    "名古屋ソニーストア"
]

# Initialize an empty dictionary to store the results
similarity_scores = {}

# Loop through each combination of keywords
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        kw1 = keywords[i]
        kw2 = keywords[j]
        # Calculate cosine similarity
        score = cosine_similarity_calc_word_mean(kw1, kw2)
        # Store the result in the dictionary
        similarity_scores[(kw1, kw2)] = score
        print(f"Cosine Similarity (word embedding mean)between '{kw1}' and '{kw2}': {score}")


Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and '一眼レフカメラ': 0.6268917322158813
Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and 'ソニー アルファ9': 0.6496142745018005
Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and 'アルファ7r': 0.4612368941307068
Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.7931433320045471
Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and '加賀屋の女将': 0.5742285251617432
Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and 'sonyカメラ新製品': 0.576417088508606
Cosine Similarity (word embedding mean)between 'ソニーのミラーレス' and '名古屋ソニーストア': 0.6739460229873657
Cosine Similarity (word embedding mean)between '一眼レフカメラ' and 'ソニー アルファ9': 0.6203197836875916
Cosine Similarity (word embedding mean)between '一眼レフカメラ' and 'アルファ7r': 0.553810715675354
Cosine Similarity (word embedding mean)between '一眼レフカメラ' and 'ソニー カメラ　ミラーレス 一眼': 0.8340797424316406
Cosine Similarity (word embedding mean)between '一眼レフカメ

In [5]:
# Initialize an empty dictionary to store the results
similarity_scores = {}

# Loop through each combination of keywords
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        kw1 = keywords[i]
        kw2 = keywords[j]
        # Calculate cosine similarity
        score = cosine_similarity_calc(kw1, kw2)
        # Store the result in the dictionary
        similarity_scores[(kw1, kw2)] = score
        print(f"Cosine Similarity between '{kw1}' and '{kw2}': {score}")

Cosine Similarity between 'ソニーのミラーレス' and 'sony ミラーレス': 0.9493904709815979
Cosine Similarity between 'ソニーのミラーレス' and 'sonyミラーレス': 0.9430749416351318
Cosine Similarity between 'ソニーのミラーレス' and 'ソニー ミラーレス 一眼': 0.9245832562446594
Cosine Similarity between 'ソニーのミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.9214127063751221
Cosine Similarity between 'ソニーのミラーレス' and 'ミラーレス ソニー': 0.9427693486213684
Cosine Similarity between 'ソニーのミラーレス' and 'ミラーレス　Sony': 0.9214802980422974
Cosine Similarity between 'ソニーのミラーレス' and 'ミラーレス　Sony': 0.9214802980422974
Cosine Similarity between 'sony ミラーレス' and 'sonyミラーレス': 0.9748486876487732
Cosine Similarity between 'sony ミラーレス' and 'ソニー ミラーレス 一眼': 0.9241964817047119
Cosine Similarity between 'sony ミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.9202791452407837
Cosine Similarity between 'sony ミラーレス' and 'ミラーレス ソニー': 0.9393922090530396
Cosine Similarity between 'sony ミラーレス' and 'ミラーレス　Sony': 0.9362118244171143
Cosine Similarity between 'sony ミラーレス' and 'ミラーレス　Sony': 0.9362118244171143
Cosine 

In [8]:
# Initialize an empty dictionary to store the results
similarity_scores = {}

# Loop through each combination of keywords
for i in range(len(keywords)):
    for j in range(i + 1, len(keywords)):
        kw1 = keywords[i]
        kw2 = keywords[j]
        # Calculate cosine similarity
        P, _, _ = scorer.score([kw1], [kw2])
        print(f"Bert Socre between '{kw1}' and '{kw2}': {P[0]}")

Bert Socre between 'ソニーのミラーレス' and 'sony ミラーレス': 0.7925456762313843
Bert Socre between 'ソニーのミラーレス' and 'sonyミラーレス': 0.8074564933776855
Bert Socre between 'ソニーのミラーレス' and 'ソニー ミラーレス 一眼': 0.8578368425369263
Bert Socre between 'ソニーのミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.8555269837379456
Bert Socre between 'ソニーのミラーレス' and 'ミラーレス ソニー': 0.8404742479324341
Bert Socre between 'ソニーのミラーレス' and 'ミラーレス　Sony': 0.7713765501976013
Bert Socre between 'ソニーのミラーレス' and 'ミラーレス　Sony': 0.7713765501976013
Bert Socre between 'sony ミラーレス' and 'sonyミラーレス': 0.9200166463851929
Bert Socre between 'sony ミラーレス' and 'ソニー ミラーレス 一眼': 0.8400001525878906
Bert Socre between 'sony ミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.8351632356643677
Bert Socre between 'sony ミラーレス' and 'ミラーレス ソニー': 0.8227881193161011
Bert Socre between 'sony ミラーレス' and 'ミラーレス　Sony': 0.8079932928085327
Bert Socre between 'sony ミラーレス' and 'ミラーレス　Sony': 0.8079932928085327
Bert Socre between 'sonyミラーレス' and 'ソニー ミラーレス 一眼': 0.7965103983879089


Bert Socre between 'sonyミラーレス' and 'ソニー カメラ　ミラーレス 一眼': 0.7938821911811829
Bert Socre between 'sonyミラーレス' and 'ミラーレス ソニー': 0.7833517789840698
Bert Socre between 'sonyミラーレス' and 'ミラーレス　Sony': 0.7730127573013306
Bert Socre between 'sonyミラーレス' and 'ミラーレス　Sony': 0.7730127573013306
Bert Socre between 'ソニー ミラーレス 一眼' and 'ソニー カメラ　ミラーレス 一眼': 0.9741199016571045
Bert Socre between 'ソニー ミラーレス 一眼' and 'ミラーレス ソニー': 0.8389328718185425
Bert Socre between 'ソニー ミラーレス 一眼' and 'ミラーレス　Sony': 0.7756262421607971
Bert Socre between 'ソニー ミラーレス 一眼' and 'ミラーレス　Sony': 0.7756262421607971
Bert Socre between 'ソニー カメラ　ミラーレス 一眼' and 'ミラーレス ソニー': 0.7842466831207275
Bert Socre between 'ソニー カメラ　ミラーレス 一眼' and 'ミラーレス　Sony': 0.7391722798347473
Bert Socre between 'ソニー カメラ　ミラーレス 一眼' and 'ミラーレス　Sony': 0.7391722798347473
Bert Socre between 'ミラーレス ソニー' and 'ミラーレス　Sony': 0.8803771138191223
Bert Socre between 'ミラーレス ソニー' and 'ミラーレス　Sony': 0.8803771138191223
Bert Socre between 'ミラーレス　Sony' and 'ミラーレス　Sony': 1.0


(tensor([0.8672]), tensor([0.8928]), tensor([0.8798]))